In [1]:
from tasks.two_digit_addition_task.arithmetic import get_token_positions, get_task

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


In [ ]:
task = get_task(hf=True,size=10000)
print("Raw input:")
print(task.raw_all_data["input"][0])
task.display_counterfactual_data() 

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from pipeline import LMPipeline
import torch

device = "cuda:1" if torch.cuda.is_available() else "cpu"

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "Qwen/Qwen2.5-0.5B"
model_name = "google/gemma-2-2b"

if "llama" in model_name:
    max_new_tokens = 1
elif "gemma" in model_name:
    max_new_tokens = 3

def checker(output_text, expected):
    if expected[0] == "0":
        expected = expected[1:]
        if max_new_tokens == 3:
            return output_text[-3:-1] == expected
    return expected == output_text


pipeline = LMPipeline(model_name, max_new_tokens=max_new_tokens, device=device, dtype=torch.float16)
pipeline.tokenizer.padding_side = "left"
batch_size = 1024
print("DEVICE:", pipeline.model.device)

print("INPUT:", task.raw_all_data["input"][0])
print("LABEL:", task.raw_all_data["label"][0])
print("PREDICTION:", pipeline.dump(pipeline.generate(task.raw_all_data["input"][0])))

task.filter(pipeline, checker, verbose=True, batch_size=batch_size)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.


DEVICE: cuda:1
INPUT: The sum of 57 and 78 is 
LABEL: 135
PREDICTION: 135


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:

token_positions = get_token_positions(pipeline, task)
input = task.sample_raw_input()
print(input)
for token_position in token_positions:
    print(token_position.highlight_selected_token(input))

The sum of 73 and 49 is 
<bos>The sum of 73 and 4**9** is 
<bos>The sum of 73 and 49 is** **


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

start = 0
end = pipeline.get_num_layers()
config={"batch_size":256 , "evaluation_batch_size": batch_size, "training_epoch":1, "n_features":16, "regularization_coefficient":0.0, "output_scores":False}
names = ["random", "ones_carry"]
train_data = [name + "_train" for name in names]
validation_data = [name + "_validation" for name in names]
test_data = [name + "_test" for name in names]
test_data += [name + "_testprivate" for name in names]
verbose = True
results_dir = "math_results"

In [ ]:
from experiments.aggregate_experiments import residual_stream_baselines
residual_stream_baselines(pipeline=pipeline, task=task, token_positions=token_positions, train_data=train_data, test_data=test_data, config=config, target_variables=["ones_carry"], checker=checker, start=start, end=end, verbose=verbose, results_dir=results_dir, 
                          methods=["DBM+SAE"])

Running interventions for random_test with model units [id='ResidualStream(Layer:0,Token:op2_last)']
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


Processing batches: 100%|██████████| 5/5 [00:40<00:00,  8.20s/it]


Running interventions for random_test with model units [id='ResidualStream(Layer:0,Token:last)']
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


Processing batches: 100%|██████████| 5/5 [00:41<00:00,  8.25s/it]


Running interventions for random_test with model units [id='ResidualStream(Layer:1,Token:op2_last)']
Intervention key: layer_1_comp_block_output_unit_pos_nunit_1#0


...

Processing batches: 100%|██████████| 5/5 [00:41<00:00,  8.29s/it]


Running interventions for ones_carry_test with model units [id='ResidualStream(Layer:4,Token:op2_last)']
Intervention key: layer_4_comp_block_output_unit_pos_nunit_1#0


Processing batches: 100%|██████████| 5/5 [00:41<00:00,  8.37s/it]


Running interventions for ones_carry_test with model units [id='ResidualStream(Layer:4,Token:last)']
Intervention key: layer_4_comp_block_output_unit_pos_nunit_1#0


Processing batches:  80%|████████  | 4/5 [00:35<00:08,  8.90s/it]